<a href="https://colab.research.google.com/github/peculab/PythonAI4Beginners/blob/main/%E7%AC%AC%E5%8D%81%E4%B8%89%E9%80%B1_%E6%96%87%E5%AD%97%E5%88%86%E9%A1%9E_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**大家為什麼會來上這門課**
https://github.com/peculab/PythonAI4Beginners/issues/1

In [37]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [38]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1kvaMMSudWLyBV3VWOe09bee9-GQFUxSQWayuVYvVTds/edit?usp=sharing').sheet1

In [ ]:
dicts = gc.open_by_url('https://docs.google.com/spreadsheets/d/1zrCPUzVIOPlAc7P3ZXT_TcfVAyXl1qO8n-gIeuOg38A/edit?usp=sharing').get_worksheet(1)
dicts = dicts.get_all_records()
dicts = pd.DataFrame(dicts)

In [39]:
# 讀取所有數據
rows = gsheets.get_all_records()
import pandas as pd
df = pd.DataFrame(rows)

In [40]:
df.head()

,Account,Date,Cleaned_Content
0,peculab,"Aug 27, 2024",Owner peculab • 學校、姓名、以及你為什麼來上這門課？並留下你的第一個 co...
1,pupupeter,"Sep 2, 2024",Collaborator pupupeter • 高中生們兒早哇，我是這次的助教，我叫家瑋...
2,Junxian0829,"Sep 2, 2024",Collaborator Junxian0829 • 各位好，我是鄭鈞賢，是台師大機電系大...
3,tedlu-tw,"Sep 2, 2024",Collaborator tedlu-tw 各位好，和大家一樣是高中生，就讀北市數位實中，...
4,Eric970308-123,"Sep 2, 2024",Eric970308-123 大家好，我是胡程勛，目前高二，就讀彰化縣溪湖高中207班，在...


In [41]:
df.describe()

,Account,Date,Cleaned_Content
count,312,312,312
unique,272,13,312
top,peculab,"Sep 5, 2024",Owner peculab • 學校、姓名、以及你為什麼來上這門課？並留下你的第一個 co...
freq,11,110,1


In [42]:
!pip install jieba
!pip install snownlp

In [43]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [44]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [ ]:
for _, row in dicts.iterrows():
    word = row['words']
    jieba.add_word(word)

In [45]:
df['Chinese_Content'] = df['Cleaned_Content'].apply(extract_chinese)

# 使用 Jieba 斷詞
df['Tokenized'] = df['Chinese_Content'].apply(lambda x: list(jieba.cut(x)))

# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]

In [46]:
word_counts = Counter(all_words)
word_counts

Counter({'學校': 5,
         '姓名': 2,
         '以及': 3,
         '什麼': 3,
         '這門': 46,
         '課並': 1,
         '留下': 1,
         '第一': 9,
         '程式': 157,
         '連結': 16,
         '想要': 29,
         '透過': 12,
         '實現': 2,
         '心中': 1,
         '想法': 1,
         '高中生': 3,
         '這次': 2,
         '助教': 7,
         '家瑋現': 1,
         '台灣師': 1,
         '範大學': 1,
         '歷史系': 1,
         '三年': 1,
         '同學': 3,
         '大便': 1,
         '小孩子': 1,
         '這是': 13,
         '大家': 145,
         '整理': 2,
         '蔡老師': 2,
         '可以': 54,
         '觀覽': 1,
         '一下': 6,
         '下面': 3,
         '那位': 1,
         '冷面': 1,
         '笑匠': 1,
         '小心': 1,
         '嘿然後有': 1,
         '時候': 3,
         '我會': 5,
         '不正': 2,
         '搜哩嘿': 1,
         '拍謝': 1,
         '黑若': 1,
         '不想': 1,
         '直接': 1,
         '電腦': 5,
         '一關': 1,
         '機此': 1,
         '失效': 1,
         '上面': 2,
         '各位': 6,
         '鄭鈞賢': 1,
       

In [47]:
# 排除停用詞（這裡提供一個簡單的停用詞列表，可根據需要擴展）
stop_words = set(['所以', '好', '因為', '大家', '的', '是', '了', '我', '也', '在', '和', '就', '不', '有', '他', '她', '你', '我們', '這個'])
filtered_word_counts = {word: count for word, count in word_counts.items() if word not in stop_words}

In [31]:
# 將詞頻轉為 DataFrame 並排序
word_freq_df = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# 繪製關鍵字統計的條形圖
fig = px.bar(word_freq_df.head(30), x='Word', y='Frequency',
             title='關鍵字出現次數統計（前20名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [32]:
!apt-get update -qq
!apt-get install -y fonts-wqy-zenhei

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-wqy-zenhei is already the newest version (0.9.45-8).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [33]:
from wordcloud import WordCloud

# 取前 300 個最常出現的詞
top_words = word_freq_df.head(300)

# 使用 WQY Zenhei 字體
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
)

# 根據詞頻生成文字雲
wordcloud.generate_from_frequencies(dict(zip(top_words['Word'], top_words['Frequency'])))

# 將文字雲轉換為圖像
wordcloud_img = wordcloud.to_array()

# 使用 Plotly 顯示文字雲
fig = px.imshow(wordcloud_img)
fig.update_layout(
    title="文字雲",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    coloraxis_showscale=False
)
fig.show()

In [34]:
# 按日期計算討論熱度
df['Date'] = pd.to_datetime(df['Date'])
heatmap_data = df.groupby('Date').size().reset_index(name='Discussion_Count')

# 繪製趨勢圖
fig = px.line(heatmap_data, x='Date', y='Discussion_Count',
              title='不同時間段的討論熱度趨勢',
              labels={'Date': '日期', 'Discussion_Count': '討論數量'},
              markers=True)

fig.update_layout(
    xaxis_title="日期",
    yaxis_title="討論數量",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [35]:
from google.colab import userdata
apikey = userdata.get('GOOGLE_API_KEY')

In [36]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = f"請摘要出這些留言的重點：{df.to_string()}"

response = model.generate_content(prompt)
print(response.text)

這些留言主要來自一門Python與AI入門課程的學生和助教，內容包含：

* **自我介紹:**  大多數留言是學生的自我介紹，包含學校、年級、班級、姓名以及選課動機。選課動機主要圍繞著對程式設計、AI的興趣，以及未來職業規劃或學業發展的考量。部分學生提到先前有接觸過Python或相關程式設計經驗。也有一位數學老師和一位非學生參與課程。

* **程式碼連結:**  幾乎每位學生都附上了自己的Colab程式連結，有些是之前完成的作業或練習，有些則是從助教處複製的程式碼。

* **助教訊息:**  助教提醒學生記得開啟程式碼的共用權限，並鼓勵跨校學生互相交流。也有一些鼓勵和安慰學生的話語。

* **學習目標:**  學生的學習目標多樣化，包含學習程式設計、了解AI技術、應用於生活、未來職業發展、提升效率等。部分學生希望利用程式設計來解決特定問題(例如：寫作業、分析資料、設計遊戲、製作學習歷程檔案)。

總體而言，這些留言展現了學生們對程式設計和AI的熱情和求知慾，以及對課程的期待。  部分留言也展現了學生們對於使用AI工具(例如ChatGPT)來輔助學習的嘗試。

